## Dinámica de Consenso

**Eduardo Carrasco**

### Estados Iniciales:

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import random

# determinamos el estado inicial en forma aleatoria, con valores binarios y rango (tamaño de la grilla)
def inicial_random(tamano):
    #estado_inicial_r = [random.randint(0, 1) for _ in range(tamano)]
    estado_inicial_r = np.random.choice([0, 1], size=tamano)
    return estado_inicial_r

In [ ]:
# con esta función determinamos un estado inicial seteado al centro.
def inicial_centrado(tamano):
    estado_inicial_s = [0] * tamano
    estado_inicial_s[tamano // 2] = 1
    return estado_inicial_s

In [ ]:
def inicial_iterado(tamano):
    estados_iniciales = []
    for i in range(tamano):
        estado = np.zeros(tamano, dtype=int)
        estado[i] = 1
        estados_iniciales.append(estado)
    return estados_iniciales

### Definición vecinos:

In [1]:
def vecinos(estado, i, r):
    tamano = len(estado)
    estado_ext = np.pad(estado, pad_width=r, mode='wrap')
    vecindad = [estado_ext[i + r + j] for j in range(-r, r + 1) if j != 0]
    return vecindad

In [2]:
def regla_totalistica(vecindad, suma_vecinos):
    suma = sum(vecindad)
    if suma in suma_vecinos:
        return 1
    else:
        return 0

In [3]:
def siguiente_generacion(estado, r, suma_vecinos):
    tamano = len(estado)
    nueva_generacion = np.zeros(tamano, dtype=int)

    for i in range(tamano):
        vecindad = vecinos(estado, i, r)
        nueva_generacion[i] = regla_totalistica(vecindad, suma_vecinos)

    return nueva_generacion

In [ ]:
def automata_celular(r, tamano, generaciones, estado_inicial, suma_vecinos):
    automata = np.zeros((generaciones, tamano), dtype=int)
    automata[0] = estado_inicial

    for t in range(1, generaciones):
        automata[t] = siguiente_generacion(automata[t - 1], r, suma_vecinos)
    
    return automata

In [ ]:
def visualizar_automata(automata):
    plt.imshow(automata, cmap='Greys', interpolation='nearest')
    print(f"  Radio: {r}")
    print(f"  Número de Generaciones: {generaciones}")
    print(f"  Tamaño de la grilla: {tamano}")
    plt.xlabel('Tamaño de la grilla')
    plt.ylabel('Generaciones')
    plt.show()

In [4]:
def guardar_automata(automata, nombre_archivo="automata.png", ruta_carpeta="0. ACU/0. ACU_combinatoria"):
    plt.imshow(automata, cmap='Greys', interpolation='nearest')
    plt.title(f"Radio: {r}\nNúmero de Generaciones: {generaciones}\nTamaño de la grilla: {tamano}\nSuma Vecinos: {suma_vecinos}")
    plt.xlabel('Tamaño de la grilla')
    plt.ylabel('Generaciones')
    
    # Crear el nombre del archivo utilizando las variables
    nombre_archivo = f"UCA_R{r}_{tamano}_r{suma_vecinos}.png"
    
    # Guardar la imagen en la misma carpeta del script con el nombre especificado
    plt.savefig(ruta_carpeta + nombre_archivo, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
# Parámetros
r = 1
tamano = 20
generaciones = 50
suma_vecinos = [1,2] #regla totalistica
estado_inicial = inicial_random(tamano) #centrado o random
#estado_inicial = [0,0,0,0,1,0,0,0,0] #en base al tamaño de la variable tamano

# Ejecución del autómata celular
automata = automata_celular(r, tamano, generaciones, estado_inicial, suma_vecinos)
visualizar_automata(automata)
guardar_automata(automata, "mi_automata.png", "0. ACU/")

## Generación Múltiple de Combinaciones:

In [5]:
def automata_celular(r, tamano, generaciones, estado_inicial, suma_vecinos):
    automata = np.zeros((generaciones, tamano), dtype=int)
    automata[0] = estado_inicial

    for t in range(1, generaciones):
        automata[t] = siguiente_generacion(automata[t - 1], r, suma_vecinos)
    
    return automata

In [6]:
def guardar_automata(automata, r, generaciones, tamano, ruta_carpeta, nombre_archivo):
    plt.imshow(automata, cmap='Greys', interpolation='nearest')
    plt.title(f"Radio: {r}\n Generaciones: {generaciones}\n Tamaño: {tamano}\n Regla:{suma_vecinos}")
    plt.xlabel('Tamaño de la grilla')
    plt.ylabel('Generaciones')
    plt.savefig(f"{ruta_carpeta}/{nombre_archivo}.png", dpi=300)
    plt.close()

In [14]:
def simular_todas_combinaciones(r, tamano, generaciones, suma_vecinos, ruta_carpeta):
    estados = [list(map(int, list(format(i, f"0{tamano}b")))) for i in range(2 ** tamano)]

    for pos in range(len(estados)):
        estado_inicial = np.array(estados[pos]).copy()
        automata = automata_celular(r, tamano, generaciones, estado_inicial, suma_vecinos)
        nombre_archivo = f"automata_r{r}_estado{pos}.png"
        guardar_automata(automata, r, generaciones, tamano, ruta_carpeta, nombre_archivo)

In [15]:
def ejecutar_y_guardar_automatas(r, tamano, generaciones, ruta_carpeta, suma_vecinos):
    estados_iniciales = simular_todas_combinaciones(r, tamano, generaciones, suma_vecinos, ruta_carpeta)

    for idx, estado_inicial in enumerate(estados_iniciales):
        automata = automata_celular(r, tamano, generaciones, estado_inicial, suma_vecinos)
        guardar_automata(automata, r, generaciones, tamano, ruta_carpeta, f"UCA_R{r}_r{suma_vecinos}_{idx}")

In [16]:
r = 1
tamano = 4
generaciones = 10
suma_vecinos = [2]
ruta_carpeta = "0. ACU/0. ACU_combinatoria"

ejecutar_y_guardar_automatas(r, tamano, generaciones, ruta_carpeta, suma_vecinos)

TypeError: 'NoneType' object is not iterable

In [ ]:
from scipy.stats import entropy
def calcular_entropia(automata):
    entropias = []
    for fila in automata:
        # Contamos la cantidad de 1 y 0 en cada fila
        unique, counts = np.unique(fila, return_counts=True)
        frecuencias = dict(zip(unique, counts))
        # Calculamos la probabilidad de 0 y 1 en cada fila
        probs = [frecuencias.get(i, 0) / len(fila) for i in [0, 1]]
        entropias.append(entropy(probs, base=2))
    return np.array(entropias)

In [ ]:
def graficar_entropia(entropias):
    plt.plot(entropias)
    plt.xlabel('Generación')
    plt.ylabel('Entropía')
    plt.title('Entropía por generación')
    plt.show()

In [ ]:
# Cálculo y gráfico de la entropía en f de las generaciones:
entropias = calcular_entropia(automata)
graficar_entropia(entropias)

In [ ]:
automata

In [ ]:
tamano = 10
estados = inicial_iterado(tamano)
for estado in estados:
    print(estado)